# SCP Toolbox Workshop

___A tutorial on generating dynamically feasible trajectories reliably and efficiently___

Monday, February 7, 2022

Rocky Mountain AAS GN&C Conference, Breckenridge, CO

<h2 style="color: black;">
    <span style="background-color: #ECBE7B; padding: 1px;">&ensp;Part 3: Dubin's Car Trajectory Optimization&ensp;</span>
</h2>

In [1]:
import Pkg
Pkg.activate("..")

## these lines are required for local installations
# Pkg.develop(path="../../SCPToolbox.jl/")
# Pkg.precompile()

using SCPToolbox
using PyPlot, Colors, LinearAlgebra

using ECOS

  Activating project at `~/GitHub/SCPToolbox_tutorial`


Solve the following trajectory generation problem:

<center>
    <img src="media/p3-dubin-car-overview.png" width="350px"/>
    <br />
    <b>Figure.</b> Setup for Dubin's car trajectory optimization.
    <br />
    <br />
</center>

* Begin at an initial state
* Travel to the target state
* Avoid a circular obstacle in the way of a straight path
* Subject to Dubin's car dynamics (aka equations of motion)

## SCP Algorithm Overview

<center>
    <br />
    <img src="media/p3-scp-diag3.png" width="750"/>
    <br />
    <b>Figure.</b> A high-level overview of SCP algorithms for trajectory optimization problems.
    <br />
</center>

The toolbox provides an API for passing the dynamics, constraints, and other parameters of a trajectory peoblm into an internal format that SCP algorithms are programmed to operate on.

Any trajectory optimization problem using the toolbox begin by initialize a `TrajectoryProblem` object.

In [2]:
pbm = TrajectoryProblem();

All subsequent toolbox API calls will reference `pbm`, modifying its internals using the particulars of your trajectory problem.

## Dubin's Car Dynamics

In general, the dynamics of any nonlinear system are given by:

\begin{equation*}
\dot x(t) = f( t, x(t), u(t), p )
\end{equation*}

* $t\in \mathbb R$: time;
* $x(t)\in \mathbb R^n$: state vector;
* $u(t)\in \mathbb R^m$: input vector (the things that we can decide on);
* $p\in \mathbb R^d$: "parameter" vector (problem specific);

### Normalizing time

The toolbox assumes that time is normalized: $t\in [0,1]$. Consider a system whose dynamics are:

\begin{equation*}
\dot x(\tau) = \tilde f( \tau, x(\tau), u(\tau)),
\end{equation*}

where $\tau\in [0, t_f]$ is our regular "wall clock" time. Then:

\begin{equation*}
\tau = t_f \cdot t.
\end{equation*}

This transformation lets us re-write the dynamics in normalized time:

\begin{equation*}
\frac{dx}{dt} = t_f \tilde f(t, x, u) \equiv f(t, x, u, p),
\end{equation*}

where $p=t_f$ and $f=p\cdot\tilde f$.

For Dubin's car:

<center>
    <img src="media/p3-dubin-car-dynamics.png" width="200px"/>
    <br />
    <b>Figure.</b> Computed Dubin's car trajectory for minimum-input usage.
    <br />
    <br />
</center>
 
Dynamics in absolute time:

\begin{equation*}
\begin{array}{rl}
\dot{x} &= v\sin(\theta), \\
\dot{y} &= v\cos(\theta), \\
\dot{\theta} &= \omega.
\end{array}
\quad\Rightarrow\quad
\begin{bmatrix}
\dot{x} \\ \dot{y} \\ \dot{\theta}
\end{bmatrix}
=
\begin{bmatrix}
v\sin(\theta) \\
v\cos(\theta) \\
\omega
\end{bmatrix} = f(x, u),
\end{equation*}

where $x=[x;~y;~\theta]$ and $u=[v;~\omega]$.

In normalized time, we simply write as before $f\leftarrow p\cdot f$ where $p$ is the parameter vector holding just one element, the final time.

Now that we know the state and input vector dimensions, we can let the toolbox know about these.

We'll solve a fixed-final-time problem, so there are no _variable_ parameters. But the toolbox requires $n,m,d\ge 1$ so we still need to set $d=1$ at least, even if we end up not using any parameters.

The API function for defining the problem dimensions is `problem_set_dims!`.

In [3]:
n, m, d = 3, 2, 1
problem_set_dims!(pbm, n, m, d)

Let's fix the time to $t_f=3$. How would we write the dynamics as a Julia function?

In [4]:
t_f = 3
f(t, x, u, p) = begin
    x, y, θ = x
    v, ω = u
    return [v*sin(θ); v*cos(θ); ω]*t_f
end;

SCP algorithms work by iteratively linearizing nonconvex elements of the problem. This means that we have to provide the algorithm with the Jacobians of $f$.

\begin{equation*}
\begin{aligned}
A(t, x, u, p) &= \nabla_x f(t, x, u, p), \\
B(t, x, u, p) &= \nabla_u f(t, x, u, p), \\
F(t, x, u, p) &= \nabla_p f(t, x, u, p).
\end{aligned}
\end{equation*}

In [5]:
A(t, x, u, p) = begin
    x, y, θ = x
    v, ω = u
    return [0 0 v*cos(θ);
            0 0 -v*sin(θ);
            0 0 0]*t_f
end

B(t, x, u, p) = begin
    x, y, θ = x
    v, ω = u
    return [sin(θ) 0;
            cos(θ) 0;
            0 1]*t_f
end

F(t, x, u, p) = begin
    return zeros(3, 1)
end;

The toolbox API function for defining the dynamics is `problem_set_dynamics!`.

In [6]:
wrap(func) = (t, k, x, u, p, pbm) -> func(t, x, u, p)
problem_set_dynamics!(pbm, wrap(f), wrap(A), wrap(B), wrap(F))

## Boundary Conditions

The initial and terminal boundary conditions on the trajectory are set by the following two constraints:
\begin{equation*}
\begin{aligned}
g_{ic}(x(0), p) &= 0, \\
g_{tc}(x(1), p) &= 0.
\end{aligned}
\end{equation*}

In the case of Dubin's car, let's just set some simple fixed start and end points:


\begin{equation*}
\begin{aligned}
\left[x(0);~ y(0);~ \theta(0)\right] &= [0;0;0], \\
\left[x(t_f);~ y(t_f);~ \theta(t_f)\right] &= [0;2;0].
\end{aligned}
\end{equation*}

In [7]:
## car doesn't reverse
x_0 = zeros(3)
x_f = [0; 2; 0]

## car reverses
# x_0 = [0; 0.5; 0]
# x_f = [0; 1.5; 0]

g_ic(x, p) = x-x_0
g_tc(x, p) = x-x_f;

Again, we need to provide the Jacobians of $g_{ic}$ and $g_{tc}$, since in general these may be nonaffine functions.

\begin{align}
H_0(x(0), p) &= \nabla_x g_{ic}(x(0), p), \\
K_0(x(0), p) &= \nabla_p g_{ic}(x(0), p), \\
H_f(x(1), p) &= \nabla_x g_{tc}(x(1), p), \\
K_f(x(1), p) &= \nabla_p g_{tc}(x(1), p).
\end{align}

When a Jacobian is not provided, the toolbox assumes that it is zero.

In [8]:
H_0(x, p) = I(3)
H_f(x, p) = I(3);

The toolbox API function for defining the boundary conditions is `problem_set_bc!`.

In [9]:
wrap(func) = (x, p, pbm) -> func(x, p)
problem_set_bc!(pbm, :ic, wrap(g_ic), wrap(H_0))
problem_set_bc!(pbm, :tc, wrap(g_tc), wrap(H_f))

## Nonconvex Obstacle Constraint

Nonconvex obstacle constraints are encoded by the following inequality constraint:

\begin{equation*}
s(t, x(t), u(t), p) \le 0,
\end{equation*}

where $s(\cdot)\in\mathbb R^{n_s}$ is a continuously differentiable multidimensional function.

A circular obstacle of radius $r_0$ centered at $c_0$ can be avoided using the following constraint:

\begin{equation*}
\|E_{xy} x - c_0\|_2^2 \ge r_0^2,
\end{equation*}

where the state is the usual $x=[x;y;\theta]$ and $E_{xy} = \begin{bmatrix}1 & 0 & 0 \\ 0 & 1 & 0\end{bmatrix}$.

This is equivalent to setting $s(t, x, u, p) = r_0^2-\|E_{xy} x-c_0\|_2^2$.

> ⚠️ **Protip**: this constraint definition neglects that a real car is not a point, and has some size itself. To keep the car's body away from the obstactly, we can "inflate" the obstacle by some additional keepout factor $\Delta r_0$:
\begin{equation*}
s(t, x, u, p) = (r_0+\Delta r_0)^2-\|E_{xy} x-c_0\|_2^2.
\end{equation*}

In [10]:
c_0 = [-0.1; 1]
r_0 = 0.35
car_width = 0.1
Δr_0 = car_width/2
E_xy = [1 0 0;0 1 0]

s(t, x, u, p) = [(r_0+Δr_0)^2-(E_xy*x-c_0)'*(E_xy*x-c_0)];

Once again, we need to define the Jacobians of $s$. You are probably used to this by now 🤓.

\begin{equation*}
\begin{aligned}
C(t, x, u, p) &= \nabla_x s(t, x, u, p), \\
D(t, x, u, p) &= \nabla_u s(t, x, u, p), \\
G(t, x, u, p) &= \nabla_p s(t, x, u, p).
\end{aligned}
\end{equation*}

Again, the toolbox assumes zero for the Jacobians that are not provided.

In [11]:
C(t, x, u, p) = reshape(2*E_xy'*(c_0-E_xy*x), 1, 3);

The toolbox API function for defining the nonconvex constraints is `problem_set_s!`.

Because of problem statement differences among SCP algorithms, this function must be provided with a symbol corresponding to the SCP algorithm type.

- `:ptr` $\rightarrow$ Penalized Trust Region (PTR)

- `:scvx` $\rightarrow$ Successive Convexification (SCvx)

- `:gusto` $\rightarrow$ Guaranteed Sequential Trajectory Optimization (GuSTO)

In this notebook, PTR and SCvx are demonstrated.

In [12]:
# alg = :scvx;
alg = :ptr;

In [13]:
wrap(func) = (t, k, x, u, p, pbm) -> func(t, x, u, p)
problem_set_s!(pbm, alg, wrap(s), wrap(C))

## Objective Function

Borrowing from the standard choice in optimal control literature, the SCP Toolbox works with an objective function in the Bolza form:

\begin{equation*}
J(x, u, p) = \phi(x(1), p) + \int_0^1 \Gamma(x(t), u(t), p) dt.\nonumber
\end{equation*}

The function $\phi(\cdot)\in\mathbb R$ defines the terminal cost, while $\Gamma(\cdot)\in\mathbb R$ defines the running cost. Both have to be convex, at most quadratic, functions.

> ℹ️ Any nonconvexity in the cost can in general be off-loaded into the constraints by defining additional state, control, and/or parameter variables.

For Dubin's car, let's solve a trajectory that minimizes average control usage. For example, we can set:

\begin{equation*}
\begin{aligned}
\Gamma(x, u, p) &= u^\top u, \\
\phi(x(1), p) &= 0.
\end{aligned}
\end{equation*}

In [14]:
Γ(x, u, p) = u'*u;

The SCP Toolbox provides the following API functions for defining the cost:
* `problem_set_terminal_cost!`: if not provided, it is assumed that $\phi\equiv 0$;
* `problem_set_running_cost!`: if not provided, it is assumed that $\Gamma\equiv 0$;

In [15]:
wrap(func) = (t, k, x, u, p, pbm) -> func(x, u, p)
problem_set_running_cost!(pbm, alg, wrap(Γ))

## Initial Trajectory Guess

SCP algorithms require an initial guess for the trajectory. This can be very simple, and does not need to be feasible.

For Dubin's car, we will use a straight line for the state and zero for the input.

Given a time $t\in [0,1]$, the state initial guess will be:

\begin{equation*}
\begin{aligned}
\bar x(t) &= (1-t) x_0 + t x_f, \\
\bar u(t) &= 0.
\end{aligned}
\end{equation*}

SCP algorithms work by discretizing the trajectory problem into $N$ temporal nodes.

The API provides a function `problem_set_guess!` for defining the initial guess, discretized into $N$ points.

For convenience, a function `straightline_interpolate` is available to define a straight line guess.

In [16]:
state_guess(N) = straightline_interpolate(x_0, x_f, N)
input_guess(N) = straightline_interpolate(zeros(2), zeros(2), N);

In [17]:
problem_set_guess!(pbm, (N, pbm) -> begin
    x = state_guess(N)
    u = input_guess(N)
    p = zeros(1)
    return x, u, p
end)

## Configuring the SCP Solver

The SCP Toolbox provides several SCP algorithm choices: `PTR`, `SCvx`, and `GuSTO`.

Each algorithm must be provided with a set of parameters that define its behavior. Most of the parameters are shared, but some are algorithm-specific.

SCP algorithm performance depends significantly on a well-chosen set of parameters, and this is problem-dependent.

A lot of intuition is documented in our [Control Systems Magazine article](https://arxiv.org/abs/2106.09125).

## Solving the Trajectory Problem

Solving the problem amounts to initializing the SCP solver with its parameters, and providing the problem definition `pbm` from the API.

## PTR

In [18]:
# PTR Parameters
N, Nsub = 11, 10
iter_max = 30
disc_method = FOH
wvc, wtr = 1e3, 1e0
feas_tol = 5e-3
ε_abs, ε_rel = 1e-5, 1e-3
q_tr = Inf
q_exit = Inf
solver, solver_options = ECOS, Dict("verbose"=>0)

pars = PTR.Parameters(N, Nsub, iter_max, disc_method, wvc, wtr, ε_abs,
                      ε_rel, feas_tol, q_tr, q_exit, solver, solver_options);

In [19]:
if alg == :ptr
    ptr_pbm = PTR.create(pars, pbm)
    sol, history = PTR.solve(ptr_pbm)
end;

k  | status   | vd    | vs    | vbc   | J         | ΔJ %      | Δx    | Δu    | Δp    | δ     | dyn | ηx    | ηu    | ηp   
---+----------+-------+-------+-------+-----------+-----------+-------+-------+-------+-------+-----+-------+-------+------
1  | OPTIMAL  | 3e-11 | 1e-01 | 1e-13 | 1.66e+01  |           | 3e-01 | 2e+00 | 0e+00 | 3e-01 | T   | 0.28  | 2.08  | 0.00 
2  | OPTIMAL  | 3e-12 | 2e-12 | 2e-14 | 8.40e+00  | 49.35     | 1e+00 | 5e+00 | 0e+00 | 1e+00 | F   | 1.29  | 5.17  | 0.00 
3  | OPTIMAL  | 8e-15 | 4e-16 | 5e-15 | 2.69e+00  | 67.91     | 8e-01 | 4e+00 | 0e+00 | 8e-01 | F   | 0.85  | 4.12  | 0.00 
4  | OPTIMAL  | 2e-12 | 3e-13 | 3e-13 | 1.13e+00  | 58.12     | 1e-01 | 3e-01 | 0e+00 | 1e-01 | T   | 0.12  | 0.33  | 0.00 
5  | OPTIMAL  | 5e-12 | 5e-13 | 5e-13 | 9.84e-01  | 12.76     | 1e-02 | 5e-02 | 0e+00 | 1e-02 | T   | 0.01  | 0.05  | 0.00 
6  | OPTIMAL  | 4e-12 | 1e-12 | 4e-13 | 9.54e-01  | 3.06      | 3e-04 | 2e-03 | 0e+00 | 3e-04 | T   | 0.00  | 0.00  | 0.00 
7  | OPT

## SCvx

In [20]:
# SCvx Parameters
N, Nsub = 11, 10
iter_max = 30
disc_method = FOH
λ = 1000.0
ρ_0 = 0.0
ρ_1 = 0.1
ρ_2 = 0.7
β_sh = 2.0
β_gr = 2.0
η_init = 1.0
η_lb = 1e-3
η_ub = 10.0
feas_tol = 5e-3
ε_abs, ε_rel = 1e-5, 1e-3
q_tr = Inf
q_exit = Inf
solver, solver_options = ECOS, Dict("verbose"=>0)
pars = SCvx.Parameters(N, Nsub, iter_max, disc_method, λ, ρ_0, ρ_1, ρ_2, β_sh, β_gr, η_init, η_lb, η_ub, ε_abs,
                       ε_rel, feas_tol, q_tr, q_exit, solver, solver_options);

In [21]:
if alg == :scvx
    scvx_pbm = SCvx.create(pars, pbm)
    sol, history = SCvx.solve(scvx_pbm)
end;

## Trajectory plots

In [22]:
include("utils/p3_dubin_plotting.jl");

In [23]:
plot_trajectory();
plt.close() # comment out this line

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /Users/abhi/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


<center>
<div class="img_container">
	<img src="media/p3_ptr_traj_forward.png" class="img_item" width=40%>
	<img src="media/p3_ptr_traj_reverse.png" class="img_item" width=40.5%>
</div>
<b>Figure.</b> PTR solutions
</center>

<center>
<div class="img_container">
	<img src="media/p3_scvx_traj_forward.png" class="img_item" width=40%>
	<img src="media/p3_scvx_traj_reverse.png" class="img_item" width=42%>
</div>
<b>Figure.</b> SCvx solutions
</center>